In [ ]:
CP_LIMIT = 2500
MODE = 'remix'

This notebook, when run alongside a local instance of PvPoke (using XAMPP or similar), allows one to test combinations of Pokémon and derive each potential team's threat score.

## Initial experimentation

### Imports

In [ ]:
import json

In [ ]:
from collections import namedtuple

In [ ]:
from selenium.webdriver import Safari
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

You may want to use a different WebDriver if you aren't using Safari on Mac. Other WebDrivers may also be less obtrusive; I don't use my laptop much nowadays so this works for me.

In [ ]:
import pandas as pd

In [ ]:
from random import sample

In [ ]:
# url = 'http://192.168.64.2/pvpoke/src/team-builder/all/1500/wigglytuff-m-0-3-4%2Cazumarill-m-0-2-1%2Cregisteel-m-0-2-1'

### Load data

In [ ]:
with open('../pvpoke/src/data/gamemaster.json', 'r') as f:
    gm = json.load(f)

In [ ]:
with open(f'../pvpoke/src/data/rankings/{MODE}/overall/rankings-{CP_LIMIT}.json') as f:
    rankings = json.load(f)

In [ ]:
with open(f'../pvpoke/src/data/overrides/{MODE}/{CP_LIMIT}.json') as f:
    overrides = json.load(f)

In [ ]:
[x for x in rankings if x['speciesId'] == 'mantine']

### Data structures

In [ ]:
Pokemon = namedtuple('Pokemon', ['name', 'fast_move', 'charged_move1', 'charged_move2'], defaults=[None])

## Functions

### Fill Pokébox

In [ ]:
def get_moveset(pokemon_name):
    try:
        mon_overrides = [x for x in overrides if x['speciesId'] == pokemon_name][0]
    except IndexError:
        mon_overrides = {}
    fast_move = mon_overrides.get('fastMove', get_preferred_fast_move(pokemon_name))
    charged_moves = mon_overrides.get('chargedMoves', get_preferred_charged_moves(pokemon_name))
    return fast_move.lower(), charged_moves[0].lower(), charged_moves[1].lower()

In [ ]:
def get_preferred_fast_move(pokemon_name):
    moves = [move for move in [species['moves']['fastMoves']
             for species in rankings if species['speciesId'] == pokemon_name][0]]
    return pd.Series({move['moveId']: move['uses'] for move in moves}).idxmax()

In [ ]:
def get_preferred_charged_moves(pokemon_name):
    moves = [move for move in [species['moves']['chargedMoves']
             for species in rankings if species['speciesId'] == pokemon_name][0]]
    return tuple(pd.Series({move['moveId']: move['uses'] for move in moves}).nlargest(2).index)

In [ ]:
def generate_pokebox(*pokemon_names : str, **pokemon_with_moves : Pokemon):
    res = {mon: Pokemon(mon, *get_moveset(mon)) for mon in pokemon_names}
    res.update(pokemon_with_moves)
    return res

### Run PvPoke query

In [ ]:
def poke_to_string(pokemon):
    poke_data = [x for x in gm['pokemon'] if x['speciesId'] == pokemon.name.lower()][0]
    if not pokemon.charged_move1 and pokemon.charged_move2:
        pokemon.charged_move1 = pokemon.charged_move2
        pokemon.charged_move2 = None
    if not pokemon.fast_move and not pokemon.charged_move1:
        moveset = get_moveset(pokemon.name)
    else:
        moveset = pokemon.fast_move, pokemon.charged_move1, pokemon.charged_move2
    fast_move = list(sorted(poke_data['fastMoves'])).index(moveset[0].upper())
    charged_move1 = list(sorted(poke_data['chargedMoves'])).index(moveset[1].upper()) + 1
    charged_move2 = list(sorted(poke_data['chargedMoves'])).index(moveset[2].upper()) + 1 if pokemon.charged_move2 else 0
    pokes_missing_return = {'lapras'}
    if pokemon.name.endswith('_shadow'):
        if pokemon.charged_move1 > 'frustration':
            charged_move1 += 1
        if pokemon.charged_move2 and pokemon.charged_move2 > 'frustration':
            charged_move2 += 1
    elif pokemon.name in gm['shadowPokemon'] and pokemon.name not in pokes_missing_return:
        if pokemon.charged_move1 > 'return':
            charged_move1 += 1
        if pokemon.charged_move2 and pokemon.charged_move2 > 'return':
            charged_move2 += 1
    return f'{pokemon.name.lower()}-m-{fast_move}-{charged_move1}-{charged_move2}'

In [ ]:
def url_from_pokes(*pokemon):
    return f'http://192.168.64.2/pvpoke/src/team-builder/{MODE}/{CP_LIMIT}/' + '%2C'.join(poke_to_string(mon) for mon in pokemon)

#### Selenium

In [ ]:
class element_is_not_empty(object):

    def __init__(self, locator):
        self.locator = locator

    def __call__(self, driver):
        element = driver.find_element(*self.locator)   # Finding the referenced element
        if element.text == '':
            return False
        else:
            return element

In [ ]:
def get_threat_score(url):
    with Safari(quiet=True) as driver:
        driver.get(url)
        wait = WebDriverWait(driver, 10, ignored_exceptions=[NoSuchElementException])
        threat_score = int(wait.until(element_is_not_empty((By.CSS_SELECTOR, '.threat-score'))).text)
    return threat_score

## Run

### Check for Pokémon with a move

In [ ]:
# test_move = 'focus_blast'
# shadow = False
# [
#     x['speciesId'] for x in gm['pokemon']
#     if (x['speciesId'].endswith('_shadow') if shadow else True)
#     if test_move.upper() in x.get('chargedMoves', [])
# ]#[0]

### Build Pokéboxes

In [ ]:
# remix = generate_pokebox(*[
#     'greedent', 'ferrothorn', 'serperior', 'drifblim',
#     'seaking', 'abomasnow', 'empoleon',
#     'beedrill', 'wigglytuff', 'machamp', 'melmetal',
#     'charizard', 'toxicroak',],
#     registeel=Pokemon('registeel', 'lock_on', 'focus_blast'),
#     lapras=Pokemon('lapras', 'ice_shard', 'surf', 'ice_beam'),
# )

In [ ]:
# gl_non_remix = generate_pokebox(*[
#     'stunfisk_galarian', 'altaria', 'skarmory', 'vigoroth',
#     'galvantula', 'bastiodon', 'jellicent', 'politoed',
#     'venusaur', 'scrafty',],
#     umbreon=Pokemon('umbreon', 'snarl', 'psychic', 'foul_play'),
#     azumarill=Pokemon('azumarill', 'bubble', 'play_rough', 'hydro_pump'),
#     marowak_alolan=Pokemon('marowak_alolan', 'fire_spin', 'bone_club', 'shadow_ball'),
# )

In [ ]:
ul_premier = generate_pokebox(
    *[
        'sylveon', 'gallade', 'snorlax', 'gyarados',
        'electivire', 'gengar', 'charizard', 'sirfetchd',
    ],
    togekiss=Pokemon('togekiss', 'charm', 'aerial_ace'),
    dragonite=Pokemon('dragonite', 'dragon_tail', 'dragon_claw', 'hurricane'),
    gardevoir_shadow=Pokemon('gardevoir_shadow', 'charm', 'synchronoise'),
)

In [ ]:
ul_total = {
    **ul_premier,
    **generate_pokebox(
        lugia=Pokemon('lugia', 'dragon_tail', 'sky_attack'),
        melmetal=Pokemon('melmetal', 'thunder_shock', 'rock_slide'),
        giratina_origin=Pokemon('giratina_origin', 'shadow_claw', 'shadow_ball'),
    )
}

In [ ]:
ul_remix = ul_total.copy()
ul_remix.pop('togekiss')
ul_remix.pop('melmetal')

In [ ]:
# gl_pokebox = {**remix, **non_remix}

### Set Pokébox

In [ ]:
pokebox = ul_remix

In [ ]:
# quick check to make sure all strings can be generated

In [ ]:
def check_pokes(*pokemon):
    for mon in pokemon:
        poke_to_string(mon)

In [ ]:
check_pokes(*pokebox.values())

In [ ]:
# quick test

In [ ]:
get_threat_score(url_from_pokes(*[pokebox[x] for x in sample(list(pokebox), 3)]))

In [ ]:
# pokebox = {
#     'stunfisk_galarian': Pokemon('stunfisk_galarian', 'mud_shot', 'rock_slide', None),
#     'altaria': Pokemon('altaria', 'dragon_breath', 'sky_attack', None),
#     'wigglytuff': Pokemon('wigglytuff', 'charm', 'ice_beam', 'play_rough'),
#     'hypno': Pokemon('hypno', 'confusion', 'focus_blast', None),
#     'umbreon': Pokemon('umbreon', 'snarl', 'psychic', None),
#     'skarmory': Pokemon('skarmory', 'air_slash', 'sky_attack', None),
#     'toxicroak': Pokemon('toxicroak', 'poison_jab', 'sludge_bomb', None),
#     'machamp': Pokemon('machamp', 'counter', 'cross_chop', 'rock_slide'),
#     'escavalier': Pokemon('escavalier', 'counter', 'drill_run', None),
#     'registeel': Pokemon('registeel', 'lock_on', 'flash_cannon', None),
# }

## Test all combinations

Execution time increases significantly with each Pokemon added to the box. There are a LOT of unique combinations for 30 Pokemon.

In [ ]:
import itertools

In [ ]:
from random import shuffle, sample

In [ ]:
teams =  list(itertools.combinations(pokebox.values(), 3))
shuffle(teams)

In [ ]:
length = len(teams)
length

In [ ]:
scores2 = pd.DataFrame()

In [ ]:
from tqdm import tqdm

In [ ]:
teams = iter(teams)

In [ ]:
for team in tqdm(teams, total=length):
    try:
        url = url_from_pokes(*team)
        score = get_threat_score(url)
        tqdm.write(str([x.name for x in team] + [score]))
        scores2 = scores2.append(pd.Series([x.name for x in team] + [score]), ignore_index=True)
    except Exception:
        scores2 = scores2.append(pd.Series([x.name for x in team] + [10000]), ignore_index=True)

In [ ]:
scores2.sort_values(3).nsmallest(20, 3)

In [ ]:
scores2.sort_values(3).to_csv('ul_remix.csv', index=False, header=None)

In [ ]:
scores.nsmallest(20, 3)

In [ ]:
print('pokemon1 | pokemon2 | pokemon3 | score')
print(' :--: | :--: | :--: | :--: ')
for row in scores2.nsmallest(20, 3).sort_values(3).astype(int, errors='ignore').iterrows():
    print(' | '.join(row[1].astype(str).tolist()))

In [ ]:
scores.nsmallest(10, [3])

In [ ]:
my_team = ['ferrothorn', 'machamp']

In [ ]:
scores[scores.apply(lambda x: all(y in x.tolist() for y in my_team), axis=1)].sort_values(3)